# seq2seq를 이용한 제목추출

## 데이터준비

In [0]:
import pandas as pd

data_path = 'drive/My Drive/tabditor/수집 데이터'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

data = pd.read_csv(data_path+'/title_content_split.csv')

In [2]:
print(data.shape)
data.head()

(8916, 6)


,Unnamed: 0,jss_id,nb_pat,title,content,between
0,0,1,1,"""교내 스포츠 경기 활성화를 위한 노력""\n\n",대학 3학년 때 교내 배구 서포터즈 활동을 했습니다. 타과생들에겐 생소할 수 있는 ...,\n대
1,1,1,1,"""스포츠를 향한 오랜 관심""\n\n",중학교 정규수업을 들으며 스포츠를 처음 접했으며 모든 종목 수행평가에서 만점을 받으...,\n중
2,2,2,1,"""매출 손실을 막기 위한 관리자가 지녀야 할 책임감""\n\n","2년 전 카페의 관리자로 근무하며, 과발주로 인해 매장의 매출 손실을 막은 경험이 ...",\n2
3,3,3,1,"""탁월한 지식 수용 능력과 응용력""\n\n",새로운 지식을 수용하는 데 탁월한 학습능력을 갖추고 있습니다.\n대학 2학년 때 가...,\n새
4,4,3,1,"""체계적인 계획을 통한 걱정 줄이기""\n\n",반면 현재에 집중하기 보다 앞선 걱정을 많이 하는 경향이 있습니다. 특히 시험을 준...,\n반


In [5]:
data[data['content'].isnull()]

,Unnamed: 0,jss_id,nb_pat,title,content,between
4323,4323,10220,2,[은밀하게 위대하게]\n\n,NaN,\n[
4334,4334,10230,2,[은밀하게 위대하게]\n\n,NaN,\n[


In [0]:
data = data.dropna(axis=0)

In [8]:
data.shape

(8914, 6)

krtpy 패키지 
- 한글을 발음 그대로 로마자로 바꿔줍니다.
- 배포된 버전은 python2에서만 작동하므로 python3에 맞게 수정해주었습니다. 

출처: https://github.com/danrasband/krtpy  
수정본: https://github.com/seongwookchun/krtpy

In [11]:
!git clone https://github.com/seongwookchun/krtpy.git

Cloning into 'krtpy'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 35 (delta 12), reused 1 (delta 0), pack-reused 10
Unpacking objects: 100% (35/35), done.


In [0]:
import re 
from krtpy import krt
pat = re.compile('[^가-힣a-zA-Z1-9 ]')

input_texts = data['content'].apply(lambda x : krt.romanize(pat.sub('',x)).lower())
target_texts = data['title'].apply(lambda x: krt.romanize('\t'+ pat.sub('',x)).lower())


for sentence in input_texts:    
    for char in sentence:
        if char not in input_characters:
            input_characters.add(char)
for sentence in target_texts:
    for char in sentence:
        if char not in target_characters:
            target_characters.add(char)


In [14]:
input_texts

0       tay.hak 3haknyen ttay kyo.nay pay.kwu se.pho.t...
1       cwunghak.kyo cengkyu.swu.ep.ul tul.u.mye su.ph...
2       2nyen cen kha.phey.uy kwanli.ca.lo kunmwu.ha.m...
3       say.lo.wun ci.sik.ul swu.yongha.nun tey thak.w...
4       panmyen hyen.cay.ey cipcwungha.ki po.ta aphsen...
                              ...                        
8911    chang.ep ci.wen kongmo.cen.ey chamka.hayss.sup...
8912     senpay.tul.uy chwu.chen.ul pat.a 13nyento ton...
8913     no.lyek.ha.nun kes i.sang.u.lo cwung.yo.han k...
8914    cenkongkwa.mok cwung so.pi.ca hayngtongpwunsek...
8915    68nyen.uy centhongkwa choy.ko.cham sangphyo.lu...
Name: content, Length: 8914, dtype: object

## 전이학습을 위한 데이터 준비 

자소서 데이터만으로 학습을 진행하면 다양한 vocab이 나올 수 없다고  판단하여 뉴스데이터를 학습시킨 가중치 파일을 이용하여 학습을 진행했습니다.

In [15]:
news_data = pd.read_csv('/content/drive/My Drive/tabditor/소스코드/기능4. 제목 추출/news_headline_text.csv')

news_data.head()

,Unnamed: 0,summary,text
0,0,"한국당, '북 주민 송환' 쟁점화 시도…""새빨간 거짓말…국가살인""",자유한국당은 13일 북한 주민 송환과 관련한 논란에 화력을 집중하며 이 문제를 낱낱...
1,1,"국방부, 구타·가혹행위로 극단선택 장병 6명 '순직' 결정",선임의 구타와 가혹행위로 스스로 목숨을 끊은 장병 6명이 뒤늦게 '순직' 판정을 받...
2,2,"북, 베트남과 밀월 과시…노동신문 ""양국관계 소중해""",북한이 김일성 주석의 베트남 방문 55주년을 계기로 양국간 친선 우호를 재차 강조했...
3,3,밀리 미 합참의장 방한…내일 44차 한미군사위원회 개최,마크 밀리 미국 합참의장이 13일 한국을 방문한다.취임 후 처음으로 일본에 이어 한...
4,4,"민주당, '청년신도시' 총선공약 검토…3기 신도시 일부 전환 가능성",더불어민주당이 내년 총선에서 '청년신도시 조성'을 핵심공약 중 하나로 내세우는 방안...


In [0]:

pat2 = re.compile('[一-龥]')
input_texts2 = [pat2.sub('',krt.romanize(i).lower()) for i in news_data['text']]
target_texts2 = [pat2.sub('',krt.romanize(i).lower()) for i in news_data['summary'].apply(lambda x: '\t'+ x + '\n')]
pat = re.compile('[^一-龥]')

input_characters = set()
target_characters = set()

for sentence in input_texts2:    
    for char in sentence:
        if pat.search(char):
            if char not in input_characters:
                input_characters.add(char)
            
for sentence in target_texts2:
    for char in sentence:
        if pat.search(char):
            if char not in target_characters:
                target_characters.add(char)

In [18]:
target_texts2[:10]

['\thankwuktang, \'pwuk cwu.min songhwan\' cayngcemhwa si.to…"say.ppal.kan ke.cismal…kwuk.ka.sal.in"\n',
 "\tkwukpangpwu, kwu.tha·ka.hok.hayng.wi.lo kuktansenthayk cangpyeng 6myeng 'swun.cik' kyelceng\n",
 '\tpwuk, pey.thu.namkwa mil.wel kwa.si…no.tongsinmwun "yangkwuk.kwankyey so.cwunghay"\n',
 '\tmilli mi hapcham.uy.cang panghan…nay.il 44cha hanmi.kwunsa.wi.wen.hoy kay.choy\n',
 "\tmin.cwu.tang, 'chengnyensinto.si' chongsenkong.yak kemtho…3ki sinto.si il.pwu cen.hwan ka.nungseng\n",
 '\tyen.hap.sa.lyengkwan "ci.so.mi.a conglyo, wu.li.ka yak.ha.tan cal.mostoyn mey.si.ci wi.hem"\n',
 "\t[mac.cangtho.lon] 'phay.su.thu.thu.layk pep.an che.li' cen.wun ko.co…ipcang.un?\n",
 "\tpang.wi.pi ki.con.hyepceng thul kkay.lye.nun mi…'kongphyenghan pwuntam' tongsang.i.mong\n",
 "\t[a.chim& cwu.yo.nyu.su] 'phay.su.thu.thu.layk' na.kyeng.wen chwul.sek\n",
 '\t[nyu.su.chey.khu｜o.nul] ma.khu milli mi hapcham.uy.cang panghan\n']

In [19]:
#고유 문자 개수 및 단어 길이가 가장 긴 값을 변수에 지정.
import numpy as np
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = int(np.mean([len(txt) for txt in input_texts]))
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 8914
Number of unique input tokens: 105
Number of unique output tokens: 61
Max sequence length for inputs: 1436
Max sequence length for outputs: 179


In [0]:
# 딕셔너리에 인덱스로 값을 변환해서 저장. 
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    for t, char in enumerate(input_text):
        if t>=1436:
            break
       
        encoder_input_data[i, t, input_token_index[char]] = 1.
    
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        
        decoder_input_data[i, t, target_token_index[char]] = 1.
     
        
        if t > 0:
           
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
           
    if t>=1436:
        break
   
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.


In [23]:
encoder_input_data.shape

(8914, 1436, 105)

In [26]:
from keras.models import Model
from keras.layers import Input,LSTM,Dense
import numpy as np
batch_size = 64
epochs = 10
latent_dim = 256  
num_samples = 10000  
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [0]:

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [28]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
model.load_weights('drive/My Drive/tabditor/소스코드/기능4. 제목 추출/s2s_jasoseo.h5')

In [0]:
from keras.callbacks import ModelCheckpoint 
cb = ModelCheckpoint('drive/My Drive/tabditor/소스코드/기능4. 제목 추출/s2s_jasoseo.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=200,
        #   callbacks=[cb],
          validation_split=0.2)
# Save model


In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [0]:
def decode_sequence(input_seq):

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))

    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [35]:
for seq_index in range(50):
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', krt.hangulize(input_texts[seq_index],toEncoding='utf-8'))
    print('Decoded sentence:', krt.hangulize(decoded_sentence,toEncoding='utf-8'))
    print('answer sentence:', krt.hangulize(data['title'][seq_index],toEncoding='utf-8'))

-
Input sentence: 대학 3학년 때 교내 배구 서포터즈 활동을 했습니다 타과생들에겐 생소할 수 있는 대학 간 스포츠 경기가 활성화 될 수 있도록 학교 내 유동인구가 많은 지역에서 간식을 나눠주며 경기일정을 홍보하였습니다 경기장을 방문하는 관객을 대상으로 다양한 행사를 실시하여 관객유치를 위해 노력하였습니다 그 결과 연속 3경기 만원 관중을 달성하는 성과를 거두었습니다아쉬운점 1
Decoded sentence: 사람 만나길 좋아했던 나                                                                                                                                                  
answer sentence: "교내 스포츠 경기 활성화를 위한 노력"


-
Input sentence: 중학교 정규수업을 들으며 스포츠를 처음 접했으며 모든 종목 수행평가에서 만점을 받으며 스포츠관련 학과에 진학하는 목표를 세웠습니다 대학 입학 후 테니스 동아리 배구동아리를 통해 대회에 참가하였고 플로어볼과 크리켓 같은 생소한 스포츠를 접하여 스포츠에 대한 다양한 지식을 얻을 수 있었습니다좋은점 1 글자수 49자74byt어
Decoded sentence: 성실함과 깐깐함                                                                                                                                                         
answer sentence: "스포츠를 향한 오랜 관심"


-
Input sentence: 2년 전 카페의 관리자로 근무하며 과발주로 인해 매장의 매출 손실을 막은 경험이 있습니다 승진한 지 얼마 되지 않은 수퍼바이저의 업무 미숙으로 인해 이틀에 1개 정도 팔리는 샐러드 상품 3개가 매장에 입고된 적이 있었습니다 유통기

KeyboardInterrupt: ignored